In [152]:
from sklearn import datasets
import pandas as pd

In [153]:
datasets.fetch*?

datasets.fetch_20newsgroups
datasets.fetch_20newsgroups_vectorized
datasets.fetch_california_housing
datasets.fetch_covtype
datasets.fetch_kddcup99
datasets.fetch_lfw_pairs
datasets.fetch_lfw_people
datasets.fetch_olivetti_faces
datasets.fetch_openml
datasets.fetch_rcv1
datasets.fetch_species_distributions

In [154]:
def sklearn_to_df(sklearn_dataset):
    df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
    df['target'] = pd.Series(sklearn_dataset.target)
    return df

In [155]:
df = sklearn_to_df(datasets.fetch_openml(name = 'blood-transfusion-service-center'))
X = df.drop('target',axis = 'columns')
y = df['target']
cat_cols = []
num_cols = X.columns

In [156]:
X,y = datasets.make_regression(n_samples= 1000)
# X = pd.DataFrame(X)
# y = pd.DataFrame(y)


In [157]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size = 0.3, random_state=1)

In [158]:
from sklearn import set_config
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler,PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LogisticRegression,LinearRegression
set_config(display='diagram')

num_proc = make_pipeline(SimpleImputer(strategy='median'), StandardScaler(),
                         PolynomialFeatures(degree = 1,interaction_only=True))

cat_proc = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='missing'),
    OneHotEncoder(handle_unknown='ignore'))


preprocessor = make_column_transformer((num_proc, num_cols),
                                       (cat_proc, cat_cols))


model = [LinearRegression()]
pipe = make_pipeline(preprocessor,model[0])
pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler()),
                                                                  ('polynomialfeatures',
                                                                   PolynomialFeatures(degree=1,
                                                                                      interaction_only=True))]),
                                                  Index(['V1', 'V2', 'V3', 'V4'], dtype='object')),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  [])])),
                ('linearregression', LinearRegression())])

In [159]:
from sklearn import ensemble, neighbors,linear_model,svm

model_list = [#regressors
                linear_model.LinearRegression(),linear_model.Lasso(),linear_model.Ridge(),
              
              #ensembling - regressors
                ensemble.RandomForestRegressor(max_depth = 3),ensemble.AdaBoostRegressor(),ensemble.GradientBoostingRegressor(),
    
              #classifiers
                linear_model.LogisticRegression(max_iter = 500), neighbors.KNeighborsClassifier(),svm.SVC(),             
    
              #ensembling - classifiers
                ensemble.RandomForestClassifier(n_estimators=500,max_depth = 3),ensemble.AdaBoostClassifier(learning_rate=0.5),ensemble.GradientBoostingClassifier()

              ]

In [160]:
rejected_models = []
results = []
print('{:30s}Train  /  Test'.format('Model'))
print('---------------------------------------------')
for model in model_list:
    #pipe = make_pipeline(preprocessor,model)
    pipe = make_pipeline(model)
    try:        
        pipe.fit(X_train,y_train)
        
        model_name = type(model).__name__
        train_results = pipe.score(X_train,y_train)
        test_results = pipe.score(X_test,y_test)
        
        results.append([model_name, train_results, test_results])
        print('{:30s}{:.4f} / {:.4f} '.format(model_name, train_results, test_results))

    except:
        rejected_models.append(type(model).__name__)

df_results = pd.DataFrame(results, columns=["Model", "Train Results","Test Results"])

Model                         Train  /  Test
---------------------------------------------
LinearRegression              1.0000 / 1.0000 
Lasso                         0.9994 / 0.9994 
Ridge                         1.0000 / 1.0000 
RandomForestRegressor         0.7017 / 0.6324 
AdaBoostRegressor             0.8499 / 0.7847 
GradientBoostingRegressor     0.9918 / 0.9187 


In [161]:
rejected_models

['LogisticRegression',
 'KNeighborsClassifier',
 'SVC',
 'RandomForestClassifier',
 'AdaBoostClassifier',
 'GradientBoostingClassifier']

In [122]:
df_results.sort_values(by = 'Test Results',ascending = False)

,Model,Train Results,Test Results
